In [1]:
!pip install plotly==4.14.1

In [2]:
import numpy as np
import pandas as pd
import plotly.figure_factory as ff
from datetime import datetime
import matplotlib.pyplot as plt

In [3]:
def read_solution(instance_filename, solution_filename):
  instance = pd.read_csv(instance_filename, delimiter=',').set_index("Cirurgia (c)")
  solution = pd.read_csv(solution_filename, delimiter=';') \
    .set_index(["Dia (d)"]) \
    .sort_values(by=["Dia (d)", "Sala (r)", "Horário (t)"])
  return instance, solution

In [4]:
def enrich_solution(sol, inst):
  sol["Horário_fim (t)"] = sol.apply(
    lambda row:
      row["Horário (t)"] + \
      inst.loc[row["Cirurgia (c)"], "Duração (tc)"],
    axis=1,
  )
  sol["Priority"] = sol.apply(
    lambda row: inst.loc[row["Cirurgia (c)"], "Prioridade (p)"],
    axis=1,
  )
  sol["Surgeon"] = sol.apply(
    lambda row: inst.loc[row["Cirurgia (c)"], "Cirurgião (h)"],
    axis=1,
  )
  return sol

In [5]:
def plot_gantt(sol, color_by):
  plot_df = sol.copy()
  plot_df.columns = ["Surgery", "Task", "Start", "Finish", "Priority", "Surgeon"]

  for day in plot_df.index.unique():
    df = plot_df.loc[[day]]
    fig = ff.create_gantt(
      df.sort_values(by="Task"),
      group_tasks=True,
      title=f"Dia {day}",
      showgrid_x=True,
      showgrid_y=True,
      show_colorbar = True,
      colors={idx: plt.cm.tab20(idx) for idx in df[color_by].unique()},
      index_col=color_by,
      task_names="Surgery"
    )
    fig.data[-1]['marker']['cmin'] = plot_df[color_by].min()
    fig.data[-1]['marker']['cmax'] = plot_df[color_by].max()
    fig.data[-2]['marker']['cmin'] = plot_df[color_by].min()
    fig.data[-2]['marker']['cmax'] = plot_df[color_by].max()
    fig['layout']['xaxis'].update({'type': 'linear'})
  
    annots = [
      dict(x=r["Start"]+1, y=df["Task"].nunique()-r["Task"], text="#"+str(r["Surgery"]), showarrow=False, font=dict(color='black'))
      for _, r in df.iterrows()
    ]

    fig['layout']['annotations'] = annots

    fig.show()

In [6]:
def plot_gantt_for_instance(instance_filename, solution_name, color_by="Surgeon"):
  inst, sol = read_solution(instance_filename, solution_name)
  sol = enrich_solution(sol, inst)
  plot_gantt(sol, color_by=color_by)

In [7]:
plot_gantt_for_instance('./Indefinidas - i8.csv', './Indefinidas - i8_sol.csv')